In [8]:
# from langchain.llms import OpenAI
from langchain_openai import OpenAI

In [6]:
import os
os.environ['OPENAI_API_KEY'] = ""

temperature value -> How creative we want out model to be <br>
It's betwen 0 and 1 <br>
0 -> model is playing safe and not taking any risks <br>
1 -> model is taking risk and might generate wrong results be very creative

In [9]:
llm = OpenAI(temperature = 0.6)

In [ ]:
text = "What is the capital of United States"
print(llm.invoke(text))

In [12]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = ''


In [30]:
from langchain import HuggingFaceHub
llm_huggingface = HuggingFaceHub(repo_id = "google/flan-t5-large", model_kwargs = {'temperature':0.6, "max_length":64})

In [31]:
output = llm_huggingface.invoke(text)
print(output)

washington dc


In [32]:
output = llm_huggingface.invoke("Write a Poem about AI")
print(output)

AI is a machine that can do anything it wants to do. It can do anything it wants to do. It can do anything it wants to do. It can do anything it wants to do. It can do anything it wants to do. It can do anything it wants to do. It can do anything


# Prompt Templates

In [33]:
from langchain.prompts import PromptTemplate